In [1]:
# Imports (numpy, pandas, matplotlib, seaborn imported automatically)
import requests
from etherscan import Etherscan
from BondingCurveNexus import config, sys_params
from BondingCurveNexus.param_functions import open_liq, wnxm_movement_per_eth
from BondingCurveNexus.sys_class import NexusSystem
from scipy.stats import lognorm
from tqdm import tqdm
from random import shuffle

In [2]:
# activate etherscan client
eth_client = Etherscan(config.etherscan_api)
eth_price_usd = float(eth_client.get_eth_last_price()['ethusd'])
eth_price_usd

1175.85

In [3]:
supply_url = 'https://api.coingecko.com/api/v3/coins/wrapped-nxm'
wnxm_supply_now = requests.get(supply_url).json()['market_data']['total_supply']
wnxm_supply_now

3001629.61399142

In [4]:
system_instance = NexusSystem(liquidity_eth=open_liq(0.5), wnxm_move_size=wnxm_movement_per_eth(500_000))

In [8]:
print(sys_params.wnxm_price_now, system_instance.book_value(),
      f'{100 * sys_params.wnxm_price_now / system_instance.book_value()}%')

0.00936449 0.022754522986656315 41.15441139105186%


# Dutch Auction Modelling

## Dutch Auction Mechanism

Assume that we start at wNXM, as new tokenomics will start slightly below; therefore the first "interesting point" will be when the ratchet hits wNXM price.

Mechanisms:
1. Uniswap v2 Curve to push price up when buys happen & down when sales happen
2. Two-sided market above book with a x% penalty for sales
3. Buys disabled below book and price discovery through ratchet mechanism

Floor at book value for buys - we don't want to allow people to buy for less than BV to avoid dilution with cheap NXM. However, to allow price to go up we also must allow sells above BV.

### Liquidity parameters

Current proposed approach to liquidity: 
- have a large opening liquidity that allows us to get to book value while draining wNXM markets (tens of thousands of ETH)
- have a low target liquidity (<5k ETH) that liquidity approaches over time
- increase liquidity towards target below BV and decrease liquidity above BV towards target, but not the other way around

In [ ]:
# opening liquidity provided
opening_liq = 25_000
# target liquidity
liq_target = 2500
# percentage of target per day that liquidity moves (additively) towards target
liq_move_perc = 0.04
# number of daily liquidity moves
liq_moves_per_day = 10

### Ratchet sizing

Ratchet speed, for consistency, should be defined in relation to book value price. This enables a linear climb towards book value instead of a percentage or fixed number of NXM liquidity.

Also defining how many times this happens per day so that the ascent can be a bit smoother in the modelling. In practice, hopefully the ratchet is streamed.

Process inside system class:
- Define percentage x% of book value that increases on a daily basis as system parameter
- Use current book value to define x% in NXM price terms
- Solve for new NXM liquidity

In [ ]:
ratchet_bv_perc = 0.04
ratchets_per_day = 10

### Buy/sell spread

Price difference between buys & sells.

- adds an x% bonus to buyers where users wishing to provide a worth of ETH will actually receive (1+x%) * a worth of NXM
- this will not impact ETH amounts in the Uni market, but instead add extra NXM

In [ ]:
spread = 0.025

### wNXM Liquidity in Open Market

Obtained from https://www.coingecko.com/en/coins/wrapped-nxm#markets, summed +2% and -2% depth across all exchanges & averaged the two. Rounded to nearest 10k (last 05/10/2022)

Average is about 260k, but typically using 500k which assumes that liquidity is higher at higher prices. Crude, to be improved.

Function converts 2%-move USD liquidity to an absolute ETH value move per 1 ETH sold/bought on open market. This ETH value only depends on current wNXM price and not on the wNXM price at the time.

In [ ]:
wnxm_move_size = wnxm_movement_per_eth(two_perc_liq_usd=500_000)
wnxm_move_size

In [ ]:
# How a wNXM buy works:
# establish current price
wnxm_open_price = sys_params.wnxm_price_now
# establish number of wNXM bought
n_wnxm = 50000
# translate amount to ETH
n_eth = n_wnxm * wnxm_open_price
# establish closing price
wnxm_close_price = wnxm_open_price + n_eth * wnxm_movement_per_eth(250_000)
print(wnxm_open_price, n_eth, wnxm_close_price, wnxm_close_price/wnxm_open_price - 1)

### NXM buy/sell sizing - Lognormal

lognormal distribution of size of a typical entry and exit in ETH

parameterised to median value being ~1 ETH, upper quartile ~3 ETH

In [ ]:
entry_shape = 1.7
entry_loc = 0
entry_scale = 1

exit_shape = 1.7
exit_loc = 0
exit_scale = 1

In [ ]:
pd.DataFrame(lognorm.rvs(s=exit_shape, loc=exit_loc, scale=exit_scale, size=100_000)).describe()

### NXM buy/sell sizing - Deterministic

To be able to properly assess the differences between individual runs, we may want to have deterministic buy & sell pressure within and outside the platform.

Size is mean of lognormal, so about 7.5 ETH.

In [ ]:
det_sell_size = 4.2

### NXM buys/sells from platform per day

Poisson distribution of number of entries & exits per day.

Lambda parameters can also be used as deterministic numbers for runs.

In [ ]:
lambda_entries = 5
lambda_exits = 5

In [ ]:
pd.DataFrame(np.random.poisson(lam=lambda_entries, size=10_000)).describe()

### wNXM drift & diffusion

Relative % change in wNXM price on open market due to random fluctuations based on a normal distribution

In [ ]:
wnxm_shifts_per_day = 1
wnxm_drift = (1+0.000)**(1/wnxm_shifts_per_day) - 1
wnxm_diffusion = (1+0.02)**(1/wnxm_shifts_per_day) - 1

In [ ]:
print(wnxm_drift, wnxm_diffusion)

## Uniswap v2 OOP Classes

1. UniPool - Base class with separate opening and target liquidities & movement towards target


2. UniOne - subclass of 1. with a one-sided market and ratchet mechanism above BV
3. UniTwo - subclass of 1. with a two-sided market with spread above BV


4. UniOneStoch - subclass of 2. that runs stochastically
5. UniTwoStoch - subclass of 3. that runs stochastically


6. UniOneDet - subclass of 2. that runs deterministically
7. UniTwoDet - subclass of 3. that runs deterministically

### UniPool Base Class

In [ ]:
# Create class with specified initial liquidity & target liquidity

class UniPool:
    # initial parameters
    def __init__(self, open_liq, target_liq):
        # start at day 0
        self.current_day = 0
        
        # set wNXM and NXM supply, Capital Pool
        self.nxm_supply = system_instance.nxm_supply
        self.wnxm_supply = wnxm_supply_now
        self.cap_pool = system_instance.cap_pool
        
        # set initial wNXM price
        self.wnxm_price = wnxm_open_price     
        # set initial ETH liquidity
        self.liquidity_eth = open_liq
        # set target liquidity
        self.target_liq = target_liq
        
        # set change in wnxm price per 1 ETH of wnxm bought/sold
        self.wnxm_move_size = wnxm_move_size
        
        # set buy/sell spread above BV. Zero by default and set in two-sided market
        self.spread = 0
        
        # set initial NXM liquidity based on opening wNXM price
        # assume that prices have converged after initial period
        self.liquidity_nxm = self.liquidity_eth / self.wnxm_price
        
        # set intial invariant 
        self.invariant = self.liquidity_eth * self.liquidity_nxm
        
        # base entries and exits - set stochasically or deterministically in subclasses
        self.base_daily_platform_buys = np.zeros(shape=model_days, dtype=int)
        self.base_daily_platform_sales = np.zeros(shape=model_days, dtype=int)
        
        # set cumulative parameters to 0
        self.eth_sold = 0
        self.eth_acquired = 0
        self.nxm_burned = 0
        self.nxm_minted = 0
        self.wnxm_removed = 0
        self.wnxm_created = 0
        
        # set tracking lists for individual instance
        self.cap_pool_prediction = [self.cap_pool]
        self.nxm_price_prediction = [self.nxm_price(action='sale')]
        self.wnxm_price_prediction = [self.wnxm_price]
        self.nxm_supply_prediction = [self.nxm_supply]
        self.wnxm_supply_prediction = [self.wnxm_supply]
        self.book_value_prediction = [self.book_value()]
        self.liquidity_nxm_prediction = [self.liquidity_nxm]
        self.liquidity_eth_prediction = [self.liquidity_eth]
        self.eth_sold_prediction = [self.eth_sold]
        self.eth_acquired_prediction = [self.eth_acquired]
        self.nxm_burned_prediction = [self.nxm_burned]
        self.nxm_minted_prediction = [self.nxm_minted]
        self.wnxm_removed_prediction = [self.wnxm_removed]
        self.wnxm_created_prediction = [self.wnxm_created]
        
        
    # system nxm price expressed in ETH
    def book_value(self):
        if self.nxm_supply == 0:
            return 0
        return self.cap_pool/self.nxm_supply
    
    def nxm_price(self, action):
        # apply spread to buys    
        if action == 'buy':
            return self.liquidity_eth / (self.liquidity_nxm * (1 - self.spread))
        # otherwise return price without spread
        return self.liquidity_eth / self.liquidity_nxm
    
    def nxm_sale_size(self):
        # defined in stoch v det subclasses - can be stochastic or deterministic 
        return 0
        
    # one sale of n_nxm NXM
    def platform_nxm_sale(self, n_nxm):
        # limit number to total NXM
        n_nxm = min(n_nxm, self.nxm_supply)
        
        # add sold NXM to pool
        self.liquidity_nxm += n_nxm
        self.nxm_supply -= n_nxm
        
        # establish new value of eth in pool
        new_eth = self.invariant / self.liquidity_nxm
        delta_eth = self.liquidity_eth - new_eth
        
        # add ETH removed and nxm burned to cumulative total, update capital pool
        self.eth_sold += delta_eth
        self.cap_pool -= delta_eth
        self.nxm_burned += n_nxm
        
        # update ETH liquidity & invariant (which will change due to spread)
        self.liquidity_eth = new_eth
        self.invariant = self.liquidity_eth * self.liquidity_nxm
        
    # one buy of n_nxm NXM, allowing for spread
    def platform_nxm_buy(self, n_nxm):
        # buys disabled below book, so user would buy wNXM on open market instead
        if round(self.nxm_price(action='buy'), 4) < round(self.book_value(), 4):
            self.wnxm_market_buy(n_wnxm=n_nxm, remove=False)
        
        # otherwise execute buy from platform
        else:
            # remove bought NXM from pool (with spread) and add actual mint to supply
            self.liquidity_nxm -= n_nxm * (1 + self.spread)
            self.nxm_supply += n_nxm

            # establish new value of eth in pool
            new_eth = self.invariant / self.liquidity_nxm
            delta_eth = new_eth - self.liquidity_eth

            # add ETH acquired and nxm minted to cumulative total, update capital pool
            self.eth_acquired += delta_eth
            self.cap_pool += delta_eth
            self.nxm_minted += n_nxm

            # update ETH liquidity
            self.liquidity_eth = new_eth
    
    def wnxm_market_buy(self, n_wnxm, remove=True):
        # limit number of wnxm bought to total supply
        n_wnxm = min(n_wnxm, self.wnxm_supply)
        
        # crude calc for ETH amount (assuming whole buy happens on opening price)
        n_eth = n_wnxm * self.wnxm_price
        
        # increase price depending on defined liquidity parameters
        self.wnxm_price += n_eth * self.wnxm_move_size
        
        # if used for arb, remove from supply
        if remove:
            self.wnxm_supply -= n_wnxm
            self.wnxm_removed += n_wnxm
    
    def wnxm_market_sell(self, n_wnxm, create=True):
        # limit number of wnxm sold to total supply
        n_wnxm = min(n_wnxm, self.wnxm_supply)
        
        # crude calc for ETH amount (assuming whole sell happens on opening price)
        n_eth = n_wnxm * self.wnxm_price
        
        # decrease price depending on defined liquidity parameters
        self.wnxm_price -= n_eth * self.wnxm_move_size
        
        # if used for arb, add to supply
        if create:
            self.wnxm_supply += n_wnxm
            self.wnxm_created += n_wnxm
    
    def arb_sale_transaction(self):
        # establish size of nxm sell
        num = self.nxm_sale_size()
        # buy from open market
        self.wnxm_market_buy(n_wnxm=num, remove=True)
        # sell to platform
        self.platform_nxm_sale(n_nxm=num)
        
    def arb_buy_transaction(self):
        # establish size of nxm buy
        num = self.nxm_sale_size()
        # buy from platform
        self.platform_nxm_buy(n_nxm=num)
        # sell to open market
        self.wnxm_market_sell(n_wnxm=num, create=True)
    
    def ratchet_up(self): 
        # establish price movement required to be relevant percentage of BV
        price_movement = self.book_value() * ratchet_bv_perc / ratchets_per_day
        
        # establish target price and cap at book value
        target_price = min(self.nxm_price(action='sale') + price_movement, self.book_value())
        
        # update NXM liquidity to reflect new price
        self.liquidity_nxm = self.liquidity_eth / target_price
        
        # update invariant
        self.invariant = self.liquidity_eth * self.liquidity_nxm

    def ratchet_down(self): 
        # not enabled by default, enabled in one-sided market
        pass        

    def liq_move(self, new_liq):
        # solve for required NXM liquidity first from current NXM price
        self.liquidity_nxm = new_liq / self.nxm_price(action='sale')
        
        # update ETH liquidity
        self.liquidity_eth = new_liq
        
        # update invariant
        self.invariant = self.liquidity_nxm * self.liquidity_eth
        
    def new_liq(self, kind):
        # move ETH liquidity towards target
        
        # if above book & above target liq, down to target at daily percentage rate (limit at target)
        # divided by number of times we're moving liquidity per day 
        if kind == 'down':
            return max(self.liquidity_eth - self.target_liq * liq_move_perc / liq_moves_per_day, self.target_liq)
        
        # if below book & below target liq, up to target at daily percentage rate (limit at target)
        # divided by number of times we're moving liquidity per day        
        if kind == 'up':
            return min(self.liquidity_eth + self.target_liq * liq_move_perc / liq_moves_per_day, self.target_liq)

    def wnxm_shift(self):
        # defined in subclasses - can be stochastic or deterministic 
        self.wnxm_price *= 1
        
    def arbitrage(self):
        # system price > wnxm_price arb
        # in two-sided market subclass, also disable sells below BV
        while self.nxm_price(action='sale') > self.wnxm_price and\
        self.nxm_supply > 0:
            self.arb_sale_transaction()
        # system price < wnxm_price arb
        # buys always disabled below book
        while self.nxm_price(action='buy') >= self.book_value() and\
        self.nxm_price(action='buy') < self.wnxm_price:
            self.arb_buy_transaction()

    def one_day_passes(self):
        # create list of events and shuffle it
        events_today = []
        events_today.extend(['ratchet'] * ratchets_per_day)
        events_today.extend(['liq_move'] * liq_moves_per_day)
        events_today.extend(['wnxm_shift'] * wnxm_shifts_per_day)
        events_today.extend(['platform_buy'] * self.base_daily_platform_buys[self.current_day])
        events_today.extend(['platform_sale'] * self.base_daily_platform_sales[self.current_day])
        shuffle(events_today)

        # LOOP THROUGH EVENTS OF DAY
        for event in events_today:
            
            #-----WNXM ARBITRAGE-----#
            # happens in between all events
            self.arbitrage()

            #-----RATCHET-----#
            if event == 'ratchet':
                # up if below BV
                if self.book_value() > self.nxm_price(action='sale'):
                    self.ratchet_up()
                # down if above BV (but ratchet_down method only enabled in one-sided subclass)
                if self.book_value() < self.nxm_price(action='sale'):
                    self.ratchet_down()

            #-----LIQUIDITY MOVE-----#
            if event == 'liq_move':
                # liquidity down towards target if price above BV & liquidity above target 
                if self.nxm_price(action='sale') > self.book_value() and\
                    self.liquidity_eth > self.target_liq:
                    self.liq_move(new_liq=self.new_liq(kind='down'))
                
                # liquidity up towards target if price below BV & liquidity below target
                if self.nxm_price(action='sale') < self.book_value() and\
                    self.liquidity_eth < self.target_liq:
                    self.liq_move(new_liq=self.new_liq(kind='up'))
            
            #-----WNXM SHIFT-----#
            if event == 'wnxm_shift':
                self.wnxm_shift()

            #-----PLATFORM BUY-----#
            # not arbitrage-driven
            if event == 'platform_buy':
                # doesn't happen if wnxm price is below platform price
                # instead a buy happens of wNXM on open market (with no wNXM destroyed)
                if self.nxm_price(action='buy') > self.wnxm_price:
                    self.wnxm_market_buy(n_wnxm=self.nxm_sale_size(), remove=False)

                # otherwise execute the buy (subject to constraints within instance method)
                else:
                    self.platform_nxm_buy(n_nxm=self.nxm_sale_size())

            #-----PLATFORM SALE-----#
            # not arbitrage-driven
            if event == 'platform_sale':
                # doesn't happen if wnxm price is above platform price
                # instead a sell happens of wNXM on open market (with no wNXM created)
                if self.nxm_price(action='sale') < self.wnxm_price:
                    self.wnxm_market_sell(n_wnxm=self.nxm_sale_size(), create=False)

                # otherwise execute the sell (subject to constraints within instance method)
                else:
                    self.platform_nxm_sale(n_nxm=self.nxm_sale_size())
        
        # append values to tracking metrics
        self.cap_pool_prediction.append(self.cap_pool)
        self.nxm_price_prediction.append(self.nxm_price(action='sale'))
        self.wnxm_price_prediction.append(self.wnxm_price)
        self.nxm_supply_prediction.append(self.nxm_supply)
        self.wnxm_supply_prediction.append(self.wnxm_supply)
        self.book_value_prediction.append(self.book_value())
        self.liquidity_nxm_prediction.append(self.liquidity_nxm)
        self.liquidity_eth_prediction.append(self.liquidity_eth)
        self.eth_sold_prediction.append(self.eth_sold)
        self.eth_acquired_prediction.append(self.eth_acquired)
        self.nxm_burned_prediction.append(self.nxm_burned)
        self.nxm_minted_prediction.append(self.nxm_minted)
        self.wnxm_removed_prediction.append(self.wnxm_removed)
        self.wnxm_created_prediction.append(self.wnxm_created)
        
        # increment day
        self.current_day += 1

### One-sided + ratchet & Two-sided markets above BV

In [ ]:
class UniOne(UniPool):
    # one-sided market above Book Value
    def __init__(self, open_liq, target_liq):
        
        # initialise all the same stuff as UniPool
        super().__init__(open_liq, target_liq)

    def ratchet_down(self): 
        # establish price movement required to be relevant percentage of BV
        price_movement = self.book_value() * ratchet_bv_perc / ratchets_per_day
        
        # establish target price and cap at book value
        target_price = max(self.nxm_price(action='sale') - price_movement, self.book_value())
        
        # update NXM liquidity to reflect new price
        self.liquidity_nxm = self.liquidity_eth / target_price
        
        # update invariant
        self.invariant = self.liquidity_eth * self.liquidity_nxm

    def platform_nxm_sale(self, n_nxm):
        # sells disabled above book, so above book user would sell wNXM on open market instead
        if self.nxm_price(action='sale') > self.book_value():
            self.wnxm_market_sell(n_wnxm=n_nxm, create=False)
        
        # otherwise, execute platform sale
        else:
            # limit number to total NXM
            n_nxm = max(0, min(n_nxm, self.nxm_supply))

            # add sold NXM to pool
            self.liquidity_nxm += n_nxm
            self.nxm_supply -= n_nxm

            # establish new value of eth in pool
            new_eth = self.invariant / self.liquidity_nxm
            delta_eth = self.liquidity_eth - new_eth

            # add ETH removed and nxm burned to cumulative total, update capital pool
            self.eth_sold += delta_eth
            self.cap_pool -= delta_eth
            self.nxm_burned += n_nxm

            # update ETH liquidity & invariant (which will change due to spread)
            self.liquidity_eth = new_eth
            self.invariant = self.liquidity_eth * self.liquidity_nxm        

    # disable sales above book value
    def arbitrage(self):
        # in one-sided market subclass, also disable sells above BV
        while  self.nxm_price(action='sale') <= self.book_value() and\
        self.nxm_price(action='sale') > self.wnxm_price and\
        self.nxm_supply > 0:
            self.arb_sale_transaction()
        # system price < wnxm_price arb
        # buys always disabled below book
        while self.nxm_price(action='buy') >= self.book_value() and\
        self.nxm_price(action='buy') < self.wnxm_price:
            self.arb_buy_transaction()

class UniTwo(UniPool):
    def __init__(self, open_liq, target_liq):
        
        # initialise all the same stuff as UniPool
        super().__init__(open_liq, target_liq)
        
        # set spread to be non-zero value
        self.spread = spread

### Stochastic vs Deterministic Classes

In [ ]:
# stochastic subclasses

class UniOneStoch(UniOne):
    def __init__(self, open_liq, target_liq):
        
        # initialise all the same stuff as UniOne
        super().__init__(open_liq, target_liq)
        
        # base entries and exits using a poisson distribution
        self.base_daily_platform_buys = np.random.poisson(
                                                lam=lambda_entries,
                                                size=model_days)
        self.base_daily_platform_sales = np.random.poisson(
                                                lam=lambda_exits,
                                                size=model_days)
    
    def nxm_sale_size(self):
        # lognormal distribution of nxm sales
        return lognorm.rvs(s=exit_shape, loc=exit_loc, scale=exit_scale) / self.nxm_price(action='sale')
    
    def wnxm_shift(self):
        # set percentage changes in wnxm price using a normal distribution
        self.wnxm_price *= (1 + np.random.normal(loc=wnxm_drift, scale=wnxm_diffusion))
        
class UniTwoStoch(UniTwo):
    def __init__(self, open_liq, target_liq):
        
        # initialise all the same stuff as UniTwo
        super().__init__(open_liq, target_liq)
        
        # base entries and exits using a poisson distribution
        self.base_daily_platform_buys = np.random.poisson(
                                                lam=lambda_entries,
                                                size=model_days)
        self.base_daily_platform_sales = np.random.poisson(
                                                lam=lambda_exits,
                                                size=model_days)
    
    def nxm_sale_size(self):
        # lognormal distribution of nxm sales
        return lognorm.rvs(s=exit_shape, loc=exit_loc, scale=exit_scale) / self.nxm_price(action='sale')
    
    def wnxm_shift(self):
        # set percentage changes in wnxm price using a normal distribution
        self.wnxm_price *= (1 + np.random.normal(loc=wnxm_drift, scale=wnxm_diffusion))

In [ ]:
# deterministic subclasses

class UniOneDet(UniOne):
    def __init__(self, open_liq, target_liq):
        
        # initialise all the same stuff as UniOne
        super().__init__(open_liq, target_liq)
        
        # base entries and exits fixed per day
        self.base_daily_platform_buys = np.full(shape=model_days, fill_value=lambda_entries, dtype=int)
        self.base_daily_platform_sales = np.full(shape=model_days, fill_value=lambda_exits, dtype=int)
    
    def nxm_sale_size(self):
        # standard deterministic size of nxm sales
        return det_sell_size / self.nxm_price(action='sale')
    
    def wnxm_shift(self):
        # no random changes in wnxm price
        self.wnxm_price *= 1
        
class UniTwoDet(UniTwo):
    def __init__(self, open_liq, target_liq):
        
        # initialise all the same stuff as UniTwo
        super().__init__(open_liq, target_liq)
        
        # standard deterministic size of nxm sales
        self.base_daily_platform_buys = np.full(shape=model_days, fill_value=lambda_entries, dtype=int)
        self.base_daily_platform_sales = np.full(shape=model_days, fill_value=lambda_exits, dtype=int)
    
    def nxm_sale_size(self):
        # standard deterministic size of nxm sales
        return det_sell_size / self.nxm_price(action='sale')
    
    def wnxm_shift(self):
        # no random changes in wnxm price
        self.wnxm_price *= 1

## Runs

### Set up Single Simulation

In [ ]:
# model days
model_days = 730

# opening liquidity
open_liq = 20_000
#target liquidity
target_liq = 2500

# amount of USD it takes to make a 2% change in NXM price
wnxm_move_size = wnxm_movement_per_eth(two_perc_liq_usd=500_000)
# opening wnxm price
wnxm_open_price = sys_params.wnxm_price_now

# number of entries & exits per day
lambda_entries = 100
lambda_exits = 100

# lognormal dist of size of entries and exits
entry_shape = 1.7
entry_loc = 0
entry_scale = 1
exit_shape = 1.7
exit_loc = 0
exit_scale = 1

# deterministic entry & exit size
det_sell_size = 4.2

sim = UniTwoStoch(open_liq=open_liq, target_liq=target_liq)
days_run = 0

for i in tqdm(range(model_days)):
    try:
        sim.one_day_passes()
        days_run += 1
    except ZeroDivisionError:
        print('Something went to Zero!')
        break
    
    
#-----GRAPHS-----#
# Destructuring initialization
fig, axs = plt.subplots(7, 2, figsize=(15,27)) # axs is a (7,2) nd-array

# Subplot
axs[0, 0].plot(range(days_run+1), sim.nxm_price_prediction)
axs[0, 0].set_title('nxm_price')
# Subplot
axs[0, 1].plot(range(days_run+1), sim.wnxm_price_prediction)
axs[0, 1].set_title('wnxm_price')
# Subplot
axs[1, 0].plot(range(days_run+1), sim.nxm_supply_prediction)
axs[1, 0].set_title('nxm_supply')
# Subplot
axs[1, 1].plot(range(days_run+1), sim.wnxm_supply_prediction)
axs[1, 1].set_title('wnxm_supply')
# Subplot
axs[2, 0].plot(range(days_run+1), sim.book_value_prediction)
axs[2, 0].set_title('book_value')
# Subplot
axs[2, 1].plot(range(days_run+1), sim.cap_pool_prediction)
axs[2, 1].set_title('cap_pool')
# Subplot
axs[3, 0].plot(range(days_run+1), sim.liquidity_nxm_prediction)
axs[3, 0].set_title('liquidity_nxm')
# Subplot
axs[3, 1].plot(range(days_run+1), sim.liquidity_eth_prediction)
axs[3, 1].plot(range(days_run+1), np.full(shape=days_run+1, fill_value=target_liq))
axs[3, 1].set_title('liquidity_eth')
# Subplot
axs[4, 0].plot(range(days_run+1), sim.nxm_burned_prediction)
axs[4, 0].set_title('nxm_burned')
# Subplot
axs[4, 1].plot(range(days_run+1), sim.nxm_minted_prediction)
axs[4, 1].set_title('nxm_minted')
# Subplot
axs[5, 0].plot(range(days_run+1), sim.eth_sold_prediction)
axs[5, 0].set_title('eth_sold')
# Subplot
axs[5, 1].plot(range(days_run+1), sim.eth_acquired_prediction)
axs[5, 1].set_title('eth_acquired')
# Subplot
axs[6, 0].plot(range(days_run+1), sim.wnxm_removed_prediction)
axs[6, 0].set_title('wnxm_removed')
# Subplot
axs[6, 1].plot(range(days_run+1), sim.wnxm_created_prediction)
axs[6, 1].set_title('wnxm_created')

plt.show()

In [ ]:
print(f'''Closing state:
nxm_price: {sim.nxm_price(action='sale')}
wnxm_price: {sim.wnxm_price}
book_value: {sim.book_value()}
cap_pool: {sim.cap_pool}
nxm_supply: {sim.nxm_supply}
wnxm_supply: {sim.wnxm_supply}
liquidity_eth: {sim.liquidity_eth}
liquidity_nxm: {sim.liquidity_nxm}
''')

### Multi-sim set-ups

In [ ]:
# model days
model_days = 730

# opening liquidity
open_liq = 20_000
#target liquidity
target_liq = 2500

# amount of USD it takes to make a 2% change in NXM price
wnxm_move_size = wnxm_movement_per_eth(two_perc_liq_usd=500_000)
# opening wnxm price
wnxm_open_price = sys_params.wnxm_price_now

# number of entries & exits per day
lambda_entries = 110
lambda_exits = 100

# lognormal dist of size of entries and exits
entry_shape = 1.7
entry_loc = 0
entry_scale = 1
exit_shape = 1.7
exit_loc = 0
exit_scale = 1

# deterministic entry & exit size
det_sell_size = 7.5

# number of simulations & initialise list of instances
num_sims = 100
sims = [UniTwoStoch(open_liq=open_liq, target_liq=target_liq) for x in range(num_sims)]

# loop through individual instances and number of days for each simulation
for sim in tqdm(sims):
    for i in range(model_days):
        sim.one_day_passes()
        
 #-----RESULT VISUALISATION-----#
# Final outcome arrays
final_cap_pool_list = [sim.cap_pool_prediction[-1] for sim in sims]
final_book_value_list = [sim.book_value_prediction[-1] for sim in sims]
final_nxm_price_list = [sim.nxm_price_prediction[-1] for sim in sims]
final_wnxm_price_list = [sim.wnxm_price_prediction[-1] for sim in sims]
final_nxm_supply_list = [sim.nxm_supply_prediction[-1] for sim in sims]
final_wnxm_supply_list = [sim.wnxm_supply_prediction[-1] for sim in sims]
final_liquidity_nxm_list = [sim.liquidity_nxm_prediction[-1] for sim in sims]
final_liquidity_eth_list = [sim.liquidity_eth_prediction[-1] for sim in sims]
final_eth_sold_list = [sim.eth_sold_prediction[-1] for sim in sims]
final_eth_acquired_list = [sim.eth_acquired_prediction[-1] for sim in sims]
final_nxm_burned_list = [sim.nxm_burned_prediction[-1] for sim in sims]
final_nxm_minted_list = [sim.nxm_minted_prediction[-1] for sim in sims]
final_wnxm_removed_list = [sim.wnxm_removed_prediction[-1] for sim in sims]
final_wnxm_created_list = [sim.wnxm_created_prediction[-1] for sim in sims]

#-----HISTOGRAMS-----#
# Destructuring initialization
fig, axs = plt.subplots(7, 2, figsize=(15,27)) # axs is a (7,2) nd-array

# Subplot
axs[0, 0].hist(final_cap_pool_list, bins=20)
axs[0, 0].set_title('cap_pool')
# Subplot
axs[0, 1].hist(final_book_value_list, bins=20)
axs[0, 1].set_title('book_value')
# Subplot
axs[1, 0].hist(final_nxm_price_list, bins=20)
axs[1, 0].set_title('nxm_price')
# Subplot
axs[1, 1].hist(final_wnxm_price_list, bins=20)
axs[1, 1].set_title('wnxm_price')
# Subplot
axs[2, 0].hist(final_nxm_supply_list, bins=20)
axs[2, 0].set_title('nxm_supply')
# Subplot
axs[2, 1].hist(final_wnxm_supply_list, bins=20)
axs[2, 1].set_title('wnxm_supply')
# Subplot
axs[3, 0].hist(final_liquidity_nxm_list, bins=20)
axs[3, 0].set_title('liquidity_nxm')
# Subplot
axs[3, 1].hist(final_liquidity_eth_list, bins=20)
axs[3, 1].set_title('liquidity_eth')
# Subplot
axs[4, 0].hist(final_eth_sold_list, bins=20)
axs[4, 0].set_title('eth_sold')
# Subplot
axs[4, 1].hist(final_eth_acquired_list, bins=20)
axs[4, 1].set_title('eth_acquired')
# Subplot
axs[5, 0].hist(final_nxm_burned_list, bins=20)
axs[5, 0].set_title('nxm_burned')
# Subplot
axs[5, 1].hist(final_nxm_minted_list, bins=20)
axs[5, 1].set_title('nxm_minted')
# Subplot
axs[6, 0].hist(final_wnxm_removed_list, bins=20)
axs[6, 0].set_title('wnxm_removed')
# Subplot
axs[6, 1].hist(final_wnxm_created_list, bins=20)
axs[6, 1].set_title('wnxm_created')

plt.show()

In [ ]:
# model days
model_days = 730

# opening liquidity
open_liq = 20_000
#target liquidity
target_liq = 2500

# amount of USD it takes to make a 2% change in NXM price
wnxm_move_size = wnxm_movement_per_eth(two_perc_liq_usd=500_000)
# opening wnxm price
wnxm_open_price = sys_params.wnxm_price_now

# number of entries & exits per day
lambda_entries = 110
lambda_exits = 100

# lognormal dist of size of entries and exits
entry_shape = 1.7
entry_loc = 0
entry_scale = 1
exit_shape = 1.7
exit_loc = 0
exit_scale = 1

# deterministic entry & exit size
det_sell_size = 7.5

# number of simulations & initialise list of instances
num_sims = 100
sims = [UniOneStoch(open_liq=open_liq, target_liq=target_liq) for x in range(num_sims)]

# loop through individual instances and number of days for each simulation
for sim in tqdm(sims):
    for i in range(model_days):
        sim.one_day_passes()
        
 #-----RESULT VISUALISATION-----#
# Final outcome arrays
final_cap_pool_list = [sim.cap_pool_prediction[-1] for sim in sims]
final_book_value_list = [sim.book_value_prediction[-1] for sim in sims]
final_nxm_price_list = [sim.nxm_price_prediction[-1] for sim in sims]
final_wnxm_price_list = [sim.wnxm_price_prediction[-1] for sim in sims]
final_nxm_supply_list = [sim.nxm_supply_prediction[-1] for sim in sims]
final_wnxm_supply_list = [sim.wnxm_supply_prediction[-1] for sim in sims]
final_liquidity_nxm_list = [sim.liquidity_nxm_prediction[-1] for sim in sims]
final_liquidity_eth_list = [sim.liquidity_eth_prediction[-1] for sim in sims]
final_eth_sold_list = [sim.eth_sold_prediction[-1] for sim in sims]
final_eth_acquired_list = [sim.eth_acquired_prediction[-1] for sim in sims]
final_nxm_burned_list = [sim.nxm_burned_prediction[-1] for sim in sims]
final_nxm_minted_list = [sim.nxm_minted_prediction[-1] for sim in sims]
final_wnxm_removed_list = [sim.wnxm_removed_prediction[-1] for sim in sims]
final_wnxm_created_list = [sim.wnxm_created_prediction[-1] for sim in sims]

#-----HISTOGRAMS-----#
# Destructuring initialization
fig, axs = plt.subplots(7, 2, figsize=(15,27)) # axs is a (7,2) nd-array

# Subplot
axs[0, 0].hist(final_cap_pool_list, bins=20)

axs[0, 0].set_title('cap_pool')
# Subplot
axs[0, 1].hist(final_book_value_list, bins=20)

axs[0, 1].set_title('book_value')
# Subplot
axs[1, 0].hist(final_nxm_price_list, bins=20)

axs[1, 0].set_title('nxm_price')
# Subplot
axs[1, 1].hist(final_wnxm_price_list, bins=20)

axs[1, 1].set_title('wnxm_price')
# Subplot
axs[2, 0].hist(final_nxm_supply_list, bins=20)

axs[2, 0].set_title('nxm_supply')
# Subplot
axs[2, 1].hist(final_wnxm_supply_list, bins=20)

axs[2, 1].set_title('wnxm_supply')
# Subplot
axs[3, 0].hist(final_liquidity_nxm_list, bins=20)

axs[3, 0].set_title('liquidity_nxm')
# Subplot
axs[3, 1].hist(final_liquidity_eth_list, bins=20)

axs[3, 1].set_title('liquidity_eth')
# Subplot
axs[4, 0].hist(final_eth_sold_list, bins=20)

axs[4, 0].set_title('eth_sold')
# Subplot
axs[4, 1].hist(final_eth_acquired_list, bins=20)

axs[4, 1].set_title('eth_acquired')
# Subplot
axs[5, 0].hist(final_nxm_burned_list, bins=20)

axs[5, 0].set_title('nxm_burned')
# Subplot
axs[5, 1].hist(final_nxm_minted_list, bins=20)

axs[5, 1].set_title('nxm_minted')
# Subplot
axs[6, 0].hist(final_wnxm_removed_list, bins=20)

axs[6, 0].set_title('wnxm_removed')
# Subplot
axs[6, 1].hist(final_wnxm_created_list, bins=20)

axs[6, 1].set_title('wnxm_created')

plt.show()

### wNXM liquidity in markets - WIP

We test a range of available wNXM liquidity in markets. At current prices, according to coingecko, a 2% shift occurs approximately at $250k sold/bought. Likely that more market sellers will be available at higher prices, so most alternative liquidity parameter tests are explored upwards.

In [ ]:
#--------FIXED VARIABLES----------#
# model days
model_days = 729

# opening liquidity
open_liq = 20_000
#target liquidity
target_liq = 2500

# opening wnxm price
wnxm_open_price = sys_params.wnxm_price_now

# number of entries & exits per day
lambda_entries = 110
lambda_exits = 100

# lognormal dist of size of entries and exits
entry_shape = 1.7
entry_loc = 0
entry_scale = 1
exit_shape = 1.7
exit_loc = 0
exit_scale = 1

# deterministic entry & exit size
det_sell_size = 4.2

#--------TESTING VARIABLES----------#
# models to be tested
models = {'One-sided model': UniOneDet(open_liq=open_liq, target_liq=target_liq),
          'Two-sided model': UniTwoDet(open_liq=open_liq, target_liq=target_liq)}
# amount of USD it takes to make a 2% change in NXM price
wnxm_liq_range_usd = [100_000, 250_000, 500_000, 1_000_000, 2_000_000]


#--------SIMULATION & LABEL LIST SET-UP---------#
sims = []
labels = []
# loop through model name strings & models themselves
for k, v in models.items():
    # loop through variable values to be tested
    for liq in wnxm_liq_range_usd:
        # establish value to instantiate class with
        wnxm_move_size = wnxm_movement_per_eth(liq)
        # append class to sims list
        sims.append(v)
        # add name to labels
        labels.append(f'{k} & 2% wNXM liquidity = {liq}')

#--------RUNNING SIMULATIONS---------#
days_run = 0
for sim in sims:
    for i in range(model_days):
        try:
            sim.one_day_passes()
            days_run += 1
        except ZeroDivisionError:
            print('Something went to Zero!')
            break  

#-----GRAPHS-----#
# Destructuring initialization
fig, axs = plt.subplots(7, 2, figsize=(15,27)) # axs is a (6,2) nd-array
# Subplot
for i in range(len(sims)):
    axs[0, 0].plot(range(days_run+1), sims[i].nxm_price_prediction, label=label_list[i])
axs[0, 0].set_title('nxm_price')
axs[0, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[0, 1].plot(range(days_run+1), sims[i].wnxm_price_prediction, label=label_list[i])
axs[0, 1].set_title('wnxm_price')
axs[0, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[1, 0].plot(range(days_run+1), sims[i].nxm_supply_prediction, label=label_list[i])
axs[1, 0].set_title('nxm_supply')
axs[1, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[1, 1].plot(range(days_run+1), sims[i].wnxm_supply_prediction, label=label_list[i])
axs[1, 1].set_title('wnxm_supply')
axs[1, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[2, 0].plot(range(days_run+1), sims[i].book_value_prediction, label=label_list[i])
axs[2, 0].set_title('book_value')
axs[2, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[2, 1].plot(range(days_run+1), sims[i].cap_pool_prediction, label=label_list[i])
axs[2, 1].set_title('cap_pool')
axs[2, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[3, 0].plot(range(days_run+1), sims[i].liquidity_nxm_prediction, label=label_list[i])
axs[3, 0].set_title('liquidity_nxm')
axs[3, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[3, 1].plot(range(days_run+1), sims[i].liquidity_eth_prediction, label=label_list[i])
axs[3, 1].set_title('liquidity_eth')
axs[3, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[4, 0].plot(range(days_run+1), sims[i].nxm_burned_prediction, label=label_list[i])
axs[4, 0].set_title('nxm_burned')
axs[4, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[4, 1].plot(range(days_run+1), sims[i].nxm_minted_prediction, label=label_list[i])
axs[4, 1].set_title('nxm_minted')
axs[4, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[5, 0].plot(range(days_run+1), sims[i].eth_sold_prediction, label=label_list[i])
axs[5, 0].set_title('eth_sold')
axs[5, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[5, 1].plot(range(days_run+1), sims[i].eth_acquired_prediction, label=label_list[i])
axs[5, 1].set_title('eth_acquired')
axs[5, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[6, 0].plot(range(days_run+1), sims[i].wnxm_removed_prediction, label=label_list[i])
axs[6, 0].set_title('wnxm_removed')
axs[6, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[6, 1].plot(range(days_run+1), sims[i].wnxm_created_prediction, label=label_list[i])
axs[6, 1].set_title('wnxm_created')
axs[6, 1].legend()

plt.show()

Again, outcome depends on wNXM price fluctuation.

**Use for other range tests:** 500k USD shifts the wNXM price by _current wNXM price in ETH * 0.02_

### Change in initial ETH supply

Test whether there is much impact in terms of changing the initial supply of ETH that is provided to the liquidity pool.

In [ ]:
# model days
model_days = 360

# opening liquidity
# liquidity_eth = 4_000

# amount of USD it takes to make a 2% change in NXM price
wnxm_move_size = wnxm_movement_per_eth(two_perc_liq_usd=500_000)
# opening NXM price
wnxm_price = sys_params.wnxm_price_now

# number of entries & exits per day
lambda_entries = 10
lambda_exits = 10

# lognormal dist of size of entries and exits
entry_shape = 2
entry_loc = 0
entry_scale = 1
exit_shape = 2
exit_loc = 0
exit_scale = 1

# deterministic entry & exit size
det_sell_size = 15

# opening liquidity options
open_liq_range = [2_000, 4_000, 7_000, 10_000, 20_000, 30_000]

sims = [UniPoolDet(liquidity_eth=liq, wnxm_move_size=wnxm_move_size, wnxm_price=wnxm_price)
        for liq in open_liq_range]

for sim in sims:
    for i in range(model_days):
        try:
            sim.one_day_passes()
            days_run += 1
        except ZeroDivisionError:
            print('Something went to Zero!')
            break

# for i in range(len(sims)):
#     plt.plot(range(model_days+1), sims[i].nxm_price_prediction, label=f'{open_liq_range[i]*100}% free cap to Uni pool')
# plt.title('nxm_price')
# plt.legend(); 
        
#-----GRAPHS-----#
# Destructuring initialization
fig, axs = plt.subplots(7, 2, figsize=(15,27)) # axs is a (6,2) nd-array
# Subplot
for i in range(len(sims)):
    axs[0, 0].plot(range(model_days+1), sims[i].nxm_price_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[0, 0].set_title('nxm_price')
axs[0, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[0, 1].plot(range(model_days+1), sims[i].wnxm_price_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[0, 1].set_title('wnxm_price')
axs[0, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[1, 0].plot(range(model_days+1), sims[i].nxm_supply_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[1, 0].set_title('nxm_supply')
axs[1, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[1, 1].plot(range(model_days+1), sims[i].wnxm_supply_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[1, 1].set_title('wnxm_supply')
axs[1, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[2, 0].plot(range(model_days+1), sims[i].book_value_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[2, 0].set_title('book_value')
axs[2, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[2, 1].plot(range(model_days+1), sims[i].cap_pool_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[2, 1].set_title('cap_pool')
axs[2, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[3, 0].plot(range(model_days+1), sims[i].liquidity_nxm_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[3, 0].set_title('liquidity_nxm')
axs[3, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[3, 1].plot(range(model_days+1), sims[i].liquidity_eth_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[3, 1].set_title('liquidity_eth')
axs[3, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[4, 0].plot(range(model_days+1), sims[i].nxm_burned_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[4, 0].set_title('nxm_burned')
axs[4, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[4, 1].plot(range(model_days+1), sims[i].nxm_minted_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[4, 1].set_title('nxm_minted')
axs[4, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[5, 0].plot(range(model_days+1), sims[i].eth_sold_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[5, 0].set_title('eth_sold')
axs[5, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[5, 1].plot(range(model_days+1), sims[i].eth_acquired_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[5, 1].set_title('eth_acquired')
axs[5, 1].legend()
# Subplot
for i in range(len(sims)):
    axs[6, 0].plot(range(model_days+1), sims[i].wnxm_removed_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[6, 0].set_title('wnxm_removed')
axs[6, 0].legend()
# Subplot
for i in range(len(sims)):
    axs[6, 1].plot(range(model_days+1), sims[i].wnxm_created_prediction, label=f'{open_liq_range[i]} ETH Opening Liq & Limit')
axs[6, 1].set_title('wnxm_created')
axs[6, 1].legend()

plt.show()